# Defining new models in TVB using the TVB DSL

In this demo we show how to use the TVB DSL in order to automatically generate model code and execute it in the TVB simulator. 
The TVB DSL is an extension of LEMS and is defined using XML syntax. 

## First let us load the necessary functions to generate code

In [17]:
import tvb.dsl.LEMS2python as templating

['', '/usr/local/lib/python3.5/dist-packages/sdict-0.1.0-py3.5.egg', '/home/sandra/Documents/HPC_TVB/tvb-root/tvb_documentation/demos', '/home/sandra/Documents/HPC_TVB/tvb-root/scientific_library', '/usr/lib/python35.zip', '/usr/lib/python3.5', '/usr/lib/python3.5/plat-x86_64-linux-gnu', '/usr/lib/python3.5/lib-dynload', '/home/sandra/.local/lib/python3.5/site-packages', '/usr/local/lib/python3.5/dist-packages', '/usr/lib/python3/dist-packages', '/home/sandra/.local/lib/python3.5/site-packages/IPython/extensions', '/home/sandra/.ipython', './NeuroML/lems', './NeuroML/lems', './NeuroML/lems', '/home/sandra/Documents/HPC_TVB/tvb-root/scientific_library/tvb/dsl/NeuroML/lems', '/home/sandra/Documents/HPC_TVB/tvb-root/scientific_library/tvb/dsl/NeuroML/lems']


ImportError: No module named 'parser.LEMS'; 'parser' is not a package

For this demo we have prepared a new model called the Montbriot model. The model has been defined using the DSL in the file <path_to_tvb_scientific_library>/tvb/dsl/NeuroML/XMLmodels.

In [5]:
xmlfile = open("../../scientific_library/tvb/dsl/NeuroML/XMLmodels/montbriot.xml","r")
model =xmlfile.read()
print(model)
xmlfile.close()

We will call the templating function in order to automatically generate the model code. This will be directly moved to the right path in the tvb library so it will be recognized by the simulator when we call it later.

In [8]:
templating.regTVB_templating('montbriot')

NameError: name 'regTVB_templating' is not defined

## Testing the new model in a tvb simulation

In [20]:
from tvb.simulator.lab import *

sim_length = 400
white_matter = connectivity.Connectivity.from_file(source_file="connectivity_68.zip")
white_matter.configure()
white_matter_coupling = coupling.Linear(a=global_coupling)
SC = self.connectivity.weights
integrator = integrators.EulerDeterministic(dt=0.1)
populations = models.Montbriot()
populations.configure()
monitorsen = (monitors.TemporalAverage(period=self.period))
sim = simulator.Simulator(model=model, connectivity=white_matter, coupling=white_matter_coupling, integrator=integrator, monitors=[monitorsen])
sim.configure()
(_,tavg_data) = sim.run(simulation_length=self.sim_length)[0]
print(tavg_data)

   INFO  log level set to INFO


ImportError: No module named 'simulator'